In [ ]:
%reset -f
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
from linearmodels.iv import IV2SLS
import functions as fun
import statsmodels.formula.api as sm
import seaborn as sns
from matplotlib import pyplot as plt

In [ ]:
df = pd.read_csv('dataset.csv')
Nobs=df['ID'].count()
df['const']=np.ones((Nobs,1))
df2 = df[df['Market share'] != 0]

df2.head(20)

In [ ]:
# Copy the dataframe
data = df2.copy().reset_index(drop=True)
data.head(20)

In [ ]:
#Scale for better intepretation

data['Price'] = data['Price']/10_000 #(Change in ms(%) for change in pris in 10.000)
data['HP'] = data['HP']/10           #(Change in ms(%) for change in HP in 10)
data['Range'] = data['Range']/10     #(Change in ms(%) for change in rækkevidde in 10)

In [ ]:
# Creating dummy for china
data['China'] = (data['Country'] == 'CN').astype(int)

In [ ]:
# Take the log of the market share
data['log_market_share'] = np.log(data['Market share'])

# OLS

In [ ]:
OLS = sm.ols('log_market_share ~ Price + Range + HP + Chargetime + China', data).fit(cov_type='HC3')
OLS.summary()#.tables[1]

# Willingness to pay

In [ ]:
alpha = OLS.params[1] # Price coefficient
beta = list(OLS.params[:1]) + list(OLS.params[2:]) # Other coefficients
beta_alpha_ratio = [b / -alpha for b in beta[1:]] #Willingness to pay (excluding constant)

for i in range(len(OLS.params.index[2:])): #[2:] to exclude constant and price
    print('W2P:', OLS.params.index[2:][i], beta_alpha_ratio[i])

25,926 DKK for every 10 increase in Range\
515 DKK for every 10 increase in HP\
-4,093 DKK for every 10 increase in Chargetime\
-691,420 DKK for Chinese cars OR -69,000 DKK for Chinese cars???

# Logit

In [ ]:
intercept = np.ones((len(data),1))
data['Intercept'] = intercept

In [ ]:
X = data[['Intercept', 'Range', 'HP', 'Chargetime', 'China']]
p_j = data['Price']

In [ ]:
# IMPORTANT: The data must be sorted by year and ID before running the function
#             Because the CCPs returned by the function are sorted by year and ID
logit_data = data.sort_values(['Year', 'ID']).reset_index(drop=True)
logit_data['CCP'] = fun.ccp(alpha, beta, data, X)

In [ ]:
logit_data['Model_year'] = logit_data['Model'] + '_' + logit_data['Year'].astype(str)
probability_ratio = fun.probability_ratio(logit_data)
probability_ratio

In [ ]:
marginal_effects = fun.marginal_effects(logit_data, OLS)
marginal_effects

In [ ]:
marginal_effects[OLS.params.index].mean()

In [ ]:
cross_marginal_effects = fun.cross_marginal_effects(logit_data, OLS)
cross_marginal_effects

In [ ]:
elasticity = fun.elasticity(logit_data['CCP'], logit_data['Model_year'], OLS.params.index, OLS.params, logit_data[OLS.params.index])
elasticity

In [ ]:
X = logit_data[OLS.params.index][1:2]
X

In [ ]:
cross_elasticity = fun.cross_elasticity(logit_data, OLS)
cross_elasticity

In [ ]:
logit_data[OLS.params.index]